# LightGBMをカスタムコンテナで利用する手順を学び、SageMakerの動作を理解します

2hを想定

コンテンツ
* カスタムコンテナ(ローカル学習、ローカル推論、学習ジョブ、推論ジョブ）
* SageMaker Training Toolkit導入（コードを外出しにする）：ローカル学習、ローカル推論


## 環境
本ノートブックは、SageMakerノートブックインスタンス上で動作確認しています。
* インスタンスタイプ：ml.t3.medium
* カーネル：conda_python3

In [1]:
import sys
#Pythonのバージョン情報
sys.version

'3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:59:51) \n[GCC 9.4.0]'

'3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:59:51) \n[GCC 9.4.0]'

In [2]:
# Pythonのバージョン確認 (システムコマンド使用
!python -V

Python 3.8.12


Python 3.8.12

In [3]:
import sagemaker

print('Current SageMaker Python SDK Version ={0}'.format(sagemaker.__version__))

Current SageMaker Python SDK Version =2.109.0


## ライブラリインポート

In [4]:
# This is a sample Python program that trains a simple LightGBM Regression model, and then performs inference.
# This implementation will work on your local computer.
#
# Prerequisites:
#   1. Install required Python packages:
#       pip install boto3 sagemaker pandas scikit-learn
#       pip install 'sagemaker[local]'
#   2. Docker Desktop has to be installed on your computer, and running.
#   3. Open terminal and run the following commands:
#       docker build  -t sagemaker-lightgbm-regression-local container/.
########################################################################################################################


In [5]:
import pandas as pd
from sagemaker.estimator import Estimator
from sagemaker.local import LocalSession
from sagemaker.predictor import csv_serializer
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [6]:
sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}

# For local training a dummy role will be sufficient
role = 'arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001'

# 1.データ準備

https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/main/lightgbm_bring_your_own_container_local_training_and_serving/lightgbm_bring_your_own_container_local_training_and_serving.py

In [7]:
data = load_boston()

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=45)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=45)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX['target'] = y_train

valX = pd.DataFrame(X_val, columns=data.feature_names)
valX['target'] = y_val

testX = pd.DataFrame(X_test, columns=data.feature_names)

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

In [8]:
from pathlib import Path

Path('./data/train').mkdir(parents=True, exist_ok=True)
Path('./data/valid').mkdir(parents=True, exist_ok=True)
Path('./data/test').mkdir(parents=True, exist_ok=True)

In [9]:
local_train = './data/train/boston_train.csv'
local_valid = './data/valid/boston_valid.csv'
local_test = './data/test/boston_test.csv'

trainX.to_csv(local_train, header=None, index=False)
valX.to_csv(local_valid, header=None, index=False)
testX.to_csv(local_test, header=None, index=False)

# 2.カスタムコンテナ作成

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-lightgbm-regression

cd container

chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


# ECRでpushしたコンテナのURIを確認

AWSコンソールでECRに移動し、作成したコンテナがあることを確認します。

image URIを取得し、以下にはりつけます。

In [ ]:
#image = 'sagemaker-lightgbm-regression-local'
#image = '805433377179.dkr.ecr.us-west-2.amazonaws.com/sagemaker-lightgbm-regression:latest' # ビルドしたイメージのURI
image = '429775515542.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-lightgbm-regression:latest'

In [ ]:
local_lightgbm = Estimator(
    image,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0})

In [ ]:
train_location = 'file://'+local_train
valid_location = 'file://'+local_valid

# ローカル学習
ECRからビルドしたイメージを持ってきて、ローカルのdockerでビルドして、実行する

In [ ]:
local_lightgbm.fit({'train':train_location, 'validation': valid_location}, logs=True)

ローカルモードの学習結果は

Amazon S3
Buckets
sagemaker-us-west-2-805433377179
sagemaker-lightgbm-regression-2022-10-03-06-17-32-054/

に出力されます。


### ローカルサービング

In [ ]:
local_predictor = local_lightgbm.deploy(1, 'local', serializer=csv_serializer) 

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = local_predictor.predict(payload).decode('utf-8')
print(predicted)

# 学習ジョブを発行

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
est_lightgbm = Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m4.2xlarge",
    hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0})

In [ ]:
import sagemaker
bucket_name = '<bucket_name>' # input your bucket name
bucket_name = 'demo-lgbm-container'

train_s3 = sagemaker.s3.S3Uploader.upload('./data/train/boston_train.csv', f's3://{bucket_name}/demo_lightgbm/train')
valid_s3 = sagemaker.s3.S3Uploader.upload('./data/valid/boston_valid.csv', f's3://{bucket_name}/demo_lightgbm/valid')

In [ ]:
est_lightgbm.fit({'train':train_s3, 'validation': valid_s3}, logs=True)

## エンドポイントにデプロイ
waitしない

In [ ]:
1

In [ ]:
predictor = est_lightgbm.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer, wait=False) 

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor.predict(payload).decode('utf-8')
print(predicted)

### 学習スクリプトをローカルに保存して実行
GitHubから実行したい場合も。

SageMaker Training Toolkitが必要

### カスタムコンテナ作成
trainは含めないように注意しましょう。

Dockerfileにて、
・train を /opt/program/train と配置
・カレントディレクトリを /opt/program に設定
・SageMaker Training Toolkit が /opt/conda/bin/train にインストールされる
・train を実行すると、カレントにある /opt/program/train が実行されてしまう。
解決するには、
・カレントディレクトリを 持ち込みのtrainがある場所にしない
・train をそもそもコンテナに入れない（確実）

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-toolkit

#cd container
cd container_smtrtoolkit ### 変更点

#chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


## 学習(ローカル)

In [ ]:
#image = '805433377179.dkr.ecr.us-west-2.amazonaws.com/sagemaker-lightgbm-toolkit:latest'
image = '429775515542.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-toolkit:latest'
#image = <input your own image URI>

In [ ]:
est_lightgbm3 = Estimator(
    image,
    role,
    instance_count=1,
    #instance_type="ml.m4.2xlarge",
    instance_type="local",
    hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0},
    #source_dir='./practice_src',
    entry_point='./src/train_practice.py'
    #entry_point='./practice_src/train_practice.sh'
    )
est_lightgbm3.fit({'train':train_s3, 'validation': valid_s3}, logs=True)

## デプロイローカル

In [ ]:
predictor3 = est_lightgbm3.deploy(1, 'local', serializer=csv_serializer) 

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor3.predict(payload).decode('utf-8')
print(predicted)

# 参考


SageMaker-Pytorth training Toolkit
https://github.com/aws/sagemaker-pytorch-training-toolkit/


SageMaker-Pytorch Inference Toolkit

https://github.com/aws/sagemaker-pytorch-inference-toolkit



https://stackoverflow.com/questions/73694705/what-is-the-difference-between-sagemaker-pytorch-training-toolkit-and-sagemaker


## 参考
SageMaker のtrainingジョブを理解する

https://github.com/aws-samples/aws-ml-jp/tree/main/sagemaker/sagemaker-traning/tutorial

# Toolkitを入れず、train からtrain.shを実行

ソースをS3に配置しなればならない


fit()について

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

datasetの指定は、S3のパスか、ローカルモードならfile://　つまりGitHubは不可

### SageMaker Traiing Toolkitについて

https://github.com/aws/sagemaker-training-toolkit/blob/master/README.md

inference toolkitもある。

https://docs.aws.amazon.com/sagemaker/latest/dg/amazon-sagemaker-toolkits.html


https://github.com/aws/sagemaker-inference-toolkit


## （おまけ）カスタムコンテナを使わず、built-inコンテナのrequirement.txtにlightgbmを記載して実行する



過去バージョン（1.3-3, 1.2-2, 1.2-1, 1.0-1)はこちら

https://github.com/aws/sagemaker-xgboost-container/releases


In [11]:
import boto3
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.5-1")
#container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "latest")

In [ ]:
container

In [ ]:
est_lightgbm5 = Estimator(
    #image,
    container, # xgboostのbuilt-inコンテナ
    role,
    instance_count=1,
    #instance_type="ml.m4.2xlarge",
    instance_type="local",
    hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0},
    source_dir='./src_builtin_container',
    entry_point='train_practice.py'
    #entry_point='./practice_src/train_practice.sh'
    )

In [ ]:
est_lightgbm5.fit({'train':train_s3, 'validation': valid_s3}, logs=True)

このコンテナでservingをするにはどうすればいいか

In [ ]:
predictor5 = est_lightgbm5.deploy(1, 'local', serializer=csv_serializer) 

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor5.predict(payload).decode('utf-8')
print(predicted)

In [19]:
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
role

In [ ]:
est_lightgbm7 = Estimator(
    #image,
    container, # xgboostのbuilt-inコンテナ
    role,
    instance_count=1,
    instance_type="ml.m4.2xlarge",
    #instance_type="local",
    hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0},
    source_dir='./src_builtin_container',
    entry_point='train_practice.py'
    #entry_point='./practice_src/train_practice.sh'
    )

In [ ]:
est_lightgbm7.fit({'train':train_s3, 'validation': valid_s3}, logs=True)

### XGBビルトインコンテナで、LGBMモデルの推論を実施

In [ ]:
est_lightgbm7

In [ ]:
predictor7 = est_lightgbm7.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer, wait=False) 

In [ ]:
predictor7

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor7.predict(payload).decode('utf-8')
print(predicted)

ローカルモードの学習、エンドポイントdeployは不可

* 学習ジョブ -> deploy endpoint
* ローカル学習 -> local deploy

XGBoostビルトインコンテナでは、LGBMの推論を実行できないので、独自にserveを指定する必要があります。

SageMaker Python SDK  
https://github.com/aws/sagemaker-python-sdk

## 実験
Sagemaker.model.Model()でコード指定できる？

https://sagemaker.readthedocs.io/en/stable/api/inference/model.html


そのモデルを指定してエンドポイントを作る？

それでビルトインコンテナを使いつつ、LGBM推論ができる？

# XGBビルトインコンテナでLGBM推論を実施するには

ライブラリのインポート
/opt/ml/model/code/requirement.txt
を配置する必要があります。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/model_server.py

既存モデルをSageMakerエンドポイントにデプロイする場合

https://dev.classmethod.jp/articles/amazon-sagemaker-deploy_existing_model/

In [12]:
image_uri = container # xgboostのbuilt-inコンテナ

print(image_uri)

354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1


In [13]:
from sagemaker.predictor import RealTimePredictor, json_deserializer


In [14]:
model_data='s3://sagemaker-ap-northeast-1-429775515542/sagemaker-xgboost-2022-10-04-06-50-15-848/output/model.tar.gz'

https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model



In [22]:
from sagemaker.model import Model

m_lgbm = Model(image_uri,
               model_data=model_data,
               role=role,
               predictor_cls=RealTimePredictor,
               #env=None,
               #name=None, 
               #vpc_config=None,
               #sagemaker_session=sess,
               #enable_network_isolation=False,
               #model_kms_key=None,
               #image_config=None, ##これなに？
               #source_dir='./src_builtin_container_inference', 
               #code_location=None, # outputされるcodeの場所
               #entry_point='serve.py',
               entry_point='./src_builtin_container_inference/serve',
               #train_practice.sh',
               #container_log_level=20,
               #dependencies=None,
               #git_config=None
              )

作成したモデルインスタンスをデプロイします

https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy

In [23]:
predictor9 = m_lgbm.deploy(#initial_instance_count=None,
              initial_instance_count=1,
              #instance_type=None,
              instance_type = 'ml.m4.xlarge',
              #serializer=None,
              #deserializer=None,
              #accelerator_type=None,
              #endpoint_name=None,
              #tags=None,
              #kms_key=None,
              wait=True,
              #data_capture_config=None,
              #async_inference_config=None,
              #serverless_inference_config=None,
              )

-----

KeyboardInterrupt: 

In [ ]:
predictor9

In [ ]:
m_lgbm

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted9 = predictor9.predict(payload).decode('utf-8')
print(predicted9)

## 参考
エンドポイントのインスタンス

https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.Predictor

# カスタムコンテナ：パターン3で、Serveを行う

まずは、serveがどのようなlogを出力するのか見極め

In [24]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-lightgbm-regression

cd container_inference

chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


Login Succeeded
Sending build context to Docker daemon  31.23kB
Step 1/10 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu
58690f9b18fc: Pulling fs layer
b51569e7c507: Pulling fs layer
da8ef40b9eca: Pulling fs layer
fb15d46c38dc: Pulling fs layer
fb15d46c38dc: Waiting
da8ef40b9eca: Verifying Checksum
da8ef40b9eca: Download complete
b51569e7c507: Verifying Checksum
b51569e7c507: Download complete
58690f9b18fc: Verifying Checksum
58690f9b18fc: Download complete
fb15d46c38dc: Verifying Checksum
fb15d46c38dc: Download complete
58690f9b18fc: Pull complete
b51569e7c507: Pull complete
da8ef40b9eca: Pull complete
fb15d46c38dc: Pull complete
Digest: sha256:91bd29a464fdabfcf44e29e1f2a5f213c6dfa750b6290e40dd6998ac79da3c41
Status: Downloaded newer image for ubuntu:16.04
 ---> b6f507652425
Step 2/10 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Running in b63bc29e3980
Removing intermediate container b63bc29e3980
 ---> 17e8b6b0ad76
Step 3/10 : ARG CONDA_DIR=/opt/conda
 ---> Running in

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [25]:
cstm_uri = '429775515542.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-lightgbm-regression:latest'

In [29]:
cstm_inf = Model(cstm_uri,
               model_data=model_data,
               role=role,
               predictor_cls=RealTimePredictor,
               #env=None,
               #name=None, 
               #vpc_config=None,
               #sagemaker_session=sess,
               #enable_network_isolation=False,
               #model_kms_key=None,
               #image_config=None, ##これなに？
               #source_dir='./src_builtin_container_inference', 
               #code_location=None, # outputされるcodeの場所
               #entry_point='serve.py',
               entry_point='./src_builtin_container_inference/serve2',
               #train_practice.sh',
               #container_log_level=20,
               #dependencies=None,
               #git_config=None
              )

In [30]:
predictor10 = cstm_inf.deploy(#initial_instance_count=None,
              initial_instance_count=1,
              #instance_type=None,
              instance_type = 'ml.m4.xlarge',
              #serializer=None,
              #deserializer=None,
              #accelerator_type=None,
              #endpoint_name=None,
              #tags=None,
              #kms_key=None,
              wait=True,
              #data_capture_config=None,
              #async_inference_config=None,
              #serverless_inference_config=None,
              )

--

KeyboardInterrupt: 

## inference toolkit 入れたバージョン

toolkit入れない場合、entry_pointは反映されなかった

In [37]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-lightgbm-regression_inftk

cd container_smintoolkit

chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


Login Succeeded
Sending build context to Docker daemon  22.53kB
Step 1/14 : FROM ubuntu:16.04
 ---> b6f507652425
Step 2/14 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 17e8b6b0ad76
Step 3/14 : ARG CONDA_DIR=/opt/conda
 ---> Using cache
 ---> c9e58a443215
Step 4/14 : ENV PATH $CONDA_DIR/bin:$PATH
 ---> Using cache
 ---> daf0b713bdde
Step 5/14 : RUN apt-get update &&     apt-get install -y --no-install-recommends         ca-certificates         cmake         build-essential         gcc         g++         git         nginx         wget &&     wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh &&     /bin/bash Miniconda3-latest-Linux-x86_64.sh -f -b -p $CONDA_DIR &&     export PATH="$CONDA_DIR/bin:$PATH" &&     conda config --set always_yes yes --set changeps1 no &&     conda install -q -y numpy scipy scikit-learn pandas flask gevent gunicorn &&     git clone --recursive --branch stable --depth 1 https://github.com/Microsoft/LightGBM && 

chmod: cannot access ‘lightgbm_regression/train’: No such file or directory
chmod: cannot access ‘lightgbm_regression/serve’: No such file or directory
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [41]:
cstm_inftk_uri = '429775515542.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-lightgbm-regression_inftk:latest'

In [45]:
cstm_inf2 = Model(cstm_inftk_uri,
               model_data=model_data,
               role=role,
               predictor_cls=RealTimePredictor,
               #env=None,
               name='aaaaaaabb', 
               #vpc_config=None,
               #sagemaker_session=sess,
               #enable_network_isolation=False,
               #model_kms_key=None,
               #image_config=None, ##これなに？
               source_dir='./src_builtin_container_inference', 
               #code_location=None, # outputされるcodeの場所
               #entry_point='serve.py',
               #entry_point='./src_builtin_container_inference/serve2',
               entry_point='serve2',
               #train_practice.sh',
               #container_log_level=20,
               #dependencies=None,
               #git_config=None
              )

In [46]:
predictor12 = cstm_inf2.deploy(#initial_instance_count=None,
              initial_instance_count=1,
              #instance_type=None,
              instance_type = 'ml.m4.xlarge',
              #serializer=None,
              #deserializer=None,
              #accelerator_type=None,
              #endpoint_name=None,
              #tags=None,
              #kms_key=None,
              wait=True,
              #data_capture_config=None,
              #async_inference_config=None,
              #serverless_inference_config=None,
              )

--

KeyboardInterrupt: 

# End Of Containts ================